### Imports

In [ ]:
import pandas as pd
import numpy as np

import missingno as msno

import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go

from scipy import stats
from scipy.stats import mannwhitneyu, shapiro

### Data Loading

In [ ]:
df_offers = pd.read_excel('../data_xlsx/df_offers.xlsx')
df_orders = pd.read_excel('../data_xlsx/df_orders.xlsx')

df_placements = pd.read_excel('../data_xlsx/df_placements.xlsx')
df_stats = pd.read_excel('../data_xlsx/df_stats.xlsx')

In [ ]:
msno.bar(df_offers)

In [ ]:
msno.bar(df_orders)

In [ ]:
msno.bar(df_placements)

In [ ]:
msno.bar(df_stats)

### Data Merging and Cleaning

In [ ]:
df_placements_with_stats = pd.merge(df_placements, df_stats, on='hash_placement_id', how='inner')

In [ ]:
df_placements_with_stats_and_category = pd.merge(df_placements_with_stats, df_offers, on='hash_offer_id', how='inner')

In [ ]:
df_placements_with_stats_and_category['CR'] = df_placements_with_stats_and_category['clicks'] / df_placements_with_stats_and_category['views']

In [ ]:
plt.hist(df_placements_with_stats_and_category['CR'], bins = 50)
plt.title('Распределение CR')
plt.xlabel('Значение')
plt.ylabel('Частота')
plt.show();

In [ ]:
data1 = df_placements_with_stats_and_category[df_placements_with_stats_and_category['views'] <= 100000]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=data1['views'],
    name='Распределение просмотров',
    opacity=0.7,
    marker_color='blue'
))

In [ ]:
data2 = df_placements_with_stats_and_category.groupby('category').agg({"views": "sum", 
                                                                       "clicks": "sum"}).reset_index()
data2['CR'] = data2['clicks'] / data2['views'] * 100
data2 = data2.sort_values('views', ascending=False)
data2

In [ ]:
len(df_placements_with_stats_and_category)

In [ ]:
df_offers_and_orders = pd.merge(df_placements_with_stats_and_category, df_orders, on = 'hash_placement_id', how='inner')

In [ ]:
published_mask = (df_offers_and_orders['reward_author'] > 0) | (df_offers_and_orders['is_published'] == True)

df_offers_and_orders_published = df_offers_and_orders[published_mask]
# будем считать, что если автор получил награду - то пост/шортс опубликован

In [ ]:
df_placements_with_stats_and_category.columns.tolist()


### IQR views

In [ ]:
def iqr(data):

    data = np.array(data)
    data = data[~np.isnan(data)]
    
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    
    # Межквартильный размах
    IQR = Q3 - Q1
    
    # Границы выбросов
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    return {
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

In [ ]:
plt.hist(df_offers_and_orders_published['views'])

In [ ]:
iqr(df_offers_and_orders_published['views'])

In [ ]:
result_v = iqr(df_offers_and_orders_published['views'])

df_offers_and_orders_v = df_offers_and_orders_published[df_offers_and_orders_published['views'] <= result_v['upper_bound']]
outliers = df_offers_and_orders_published[df_offers_and_orders_published['views'] > result_v['lower_bound']]

In [ ]:
plt.boxplot(df_offers_and_orders_v['views'])

In [ ]:
plt.boxplot(outliers['views'])

### Grouping by category

In [ ]:
data2 = df_offers_and_orders_published.groupby('category').agg({'hash_placement_id': 'count',
                                                                'views': 'sum',
                                                                'clicks': 'sum'}).reset_index()

In [ ]:
data2 = data2.sort_values('hash_placement_id', ascending = False)
data2['CR'] = data2['clicks'] / data2['views']

data2['views_per_placement'] = data2['views'] / data2['hash_placement_id']

In [ ]:
data2

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data2['category'],
    y=data2['views_per_placement'],
    marker_color='#0076fe',
    marker_line_color='#0076fe',
    marker_line_width=1.5,
    opacity=1
))

fig.update_layout(
    title='Среднее число просмотров на публикации для каждой категории',
    title_x=0.5,
    xaxis_title='Категории товаров',
    yaxis_title='',
    xaxis_tickangle=-45,
    height=600,
    showlegend=False,
    plot_bgcolor='#eaf2f8',
    paper_bgcolor='#eaf2f8'
)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data2['category'],
    y=data2['CR'],
    marker_color='#0076fe',
    marker_line_color='#0076fe',
    marker_line_width=1.5,
    opacity=1
))

fig.update_layout(
    title='Среднее число кликов на публикацию',
    title_x=0.5,
    xaxis_title='Категории товаров',
    yaxis_title='',
    xaxis_tickangle=-45,
    height=600,
    showlegend=False,
    plot_bgcolor='#eaf2f8',
    paper_bgcolor='#eaf2f8'
)

### Boxplots on price by categories

In [ ]:
cat = ['Красота и здоровье',
        'Гардероб',
        'Дом и дача',
        'Спорт и отдых',
        'Детские товары',
        'Товары для животных',
        'Канцелярские товары',
        'Продукты питания',
        'Ремонт и строительство',
        'Все товары',
        'Бытовая техника',
        'Электроника',
        'Хобби и развлечения',
        'Транспорт',
        'Товары для геймеров']

df_offers_and_orders_published_iqr = pd.DataFrame()

for group in cat:
    group_data = df_offers_and_orders_published[df_offers_and_orders_published['category'] == group]

    result_gd = iqr(group_data['price'])
    
    group_data = group_data[(group_data['price'] <= result_gd['upper_bound']) & (group_data['price'] >= result_gd['lower_bound'])]
        
    df_offers_and_orders_published_iqr = pd.concat([df_offers_and_orders_published_iqr, group_data], ignore_index=True)
 
    

In [ ]:
fig = go.Figure()

for group in cat:
    group_data = df_offers_and_orders_published_iqr[df_offers_and_orders_published_iqr['category'] == group]["price"]
    
    fig.add_trace(go.Box(
        y=group_data,
        name=str(group),
        boxpoints='outliers',
        marker_color='darkorchid',
        line_color="darkorchid",
        
    ))

fig.update_layout(
    title='Распределение стоимости опубликованных товаров',
    title_x=0.5,
    xaxis_title='Категории товаров',
    yaxis_title='Цена',
    xaxis_tickangle=-45,
    height=600,
    showlegend=True,
    plot_bgcolor='#eaf2f8',
    paper_bgcolor='#eaf2f8'
)

### Scatters

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=5, cols=3,
    subplot_titles=cat,
    vertical_spacing=0.08,
    horizontal_spacing=0.05
)

for i, group in enumerate(cat):
    group_data = df_offers_and_orders_published_iqr[df_offers_and_orders_published_iqr['category'] == group]

    if len(group_data) == 0:
        continue

    row = i // 3 + 1
    col = i % 3 + 1

    x_values = group_data['price'].values
    y_values = group_data['clicks'].values

    fig.add_trace(
        go.Scatter(
            x=x_values,
            y=y_values,
            mode='markers',
            name=group,
            marker=dict(
                color='deeppink',
                size=6,
                opacity=0.7,
                line=dict(
                    color='darkred',
                    width=0.5
                )
            ),
            showlegend=False
        ),
        row=row, col=col
    )
    
fig.update_layout(
    title_text='Price and clicks',
    title_x=0.5,
    height=1200,
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()

### Offers, that was no published

In [ ]:
offers_p_id = df_offers_and_orders_published['hash_offer_id'].values.tolist()

In [ ]:
not_published_mask = (df_offers['hash_offer_id'].isin(offers_p_id))

df_offers_not_published = df_offers[~not_published_mask]
df_offers_published = df_offers[not_published_mask]

In [ ]:
# Проверка на нормальность
print("Тест Шапиро-Уилка на нормальность:")
print(f"Опубликованные: p-value = {shapiro(df_offers_published['price'])[1]:.4f}")
print(f"Неопубликованные: p-value = {shapiro(df_offers_not_published['price'])[1]:.4f}")

In [ ]:
u_stat, p_value_u = mannwhitneyu(df_offers_published['price'], df_offers_not_published['price'], alternative='greater')
print(f"U-тест Манна-Уитни: U = {u_stat:.4f}, p-value = {p_value_u:.4f}")

In [ ]:
print("Опубликованные offers:")
print(f"Количество: {len(df_offers_published['price'])}")
print(f"Средняя цена: {df_offers_published['price'].mean():.2f}")
print(f"Медиана: {df_offers_published['price'].median():.2f}")
print(f"Стандартное отклонение: {df_offers_published['price'].std():.2f}")

print("\nНеопубликованные offers:")
print(f"Количество: {len(df_offers_not_published['price'])}")
print(f"Средняя цена: {df_offers_not_published['price'].mean():.2f}")
print(f"Медиана: {df_offers_not_published['price'].median():.2f}")
print(f"Стандартное отклонение: {df_offers_not_published['price'].std():.2f}")

### Bootstrap and XGboost

In [48]:
import numpy as np

def bootstrap_median(data, n_bootstrap=10000):
    
    n = len(data)
    boot_medians = []
    
    for _ in range(n_bootstrap):
        
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        boot_medians.append(np.median(bootstrap_sample))
    
    return np.percentile(boot_medians, [2.5, 97.5])

ci_median_published = bootstrap_median(df_offers_published['price'])
ci_median_not_published = bootstrap_median(df_offers_not_published['price'])

print("Bootstrap 95% доверительные интервалы для медиан:")
print(f"Опубликованные: {ci_median_published[0]:.2f} - {ci_median_published[1]:.2f}")
print(f"Неопубликованные: {ci_median_not_published[0]:.2f} - {ci_median_not_published[1]:.2f}")

if ci_median_published[1] < ci_median_not_published[0] or ci_median_not_published[1] < ci_median_published[0]:
    print("Доверительные интервалы не перекрываются - различия статистически значимы")
else:
    print("Доверительные интервалы перекрываются - различия могут быть не значимы")

Bootstrap 95% доверительные интервалы для медиан:
Опубликованные: 4564.95 - 5037.00
Неопубликованные: 7529.00 - 7710.00
Доверительные интервалы не перекрываются - различия статистически значимы


In [60]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df_placements_csv = pd.read_csv('../data_csv/df_placements.csv', sep=';')

df_offers_csv = pd.read_csv('../data_csv/df_offers.csv', sep=';')

df_stats_csv = pd.read_csv('../data_csv/df_stats.csv', sep=';')

print("Starting feature engineering...")

data = pd.merge(df_stats_csv, df_placements_csv, on='hash_placement_id', how='left')
data = pd.merge(data, df_offers_csv, on='hash_offer_id', how='left')

data['views'] = data['views'].apply(lambda x: eval(x)[0])
data['clicks'] = data['clicks'].apply(lambda x: eval(x)[0])

data['price'].fillna('0,0', inplace=True)
data['price'] = data['price'].apply(lambda x: eval(str(x))[0])

data['placement_format_x'] = data['placement_format_x'].astype('category')
data['hash_placement_id'] = data['hash_placement_id'].astype('category')
data['placement_created_at'] = data['placement_created_at'].astype('category')
data['published_at'] = data['published_at'].astype('category')
data['offer_created_at'] = data['offer_created_at'].astype('category')

data['ctr'] = data['clicks'] / data['views']

author_stats = data.groupby('hash_author_id').agg(
    author_avg_ctr=('ctr', 'mean'),
    author_placement_count=('hash_offer_id', 'count')
).reset_index()
data = pd.merge(data, author_stats, on='hash_author_id', how='left')

offer_stats = data.groupby('hash_offer_id').agg(
    offer_avg_ctr=('ctr', 'mean'),
    offer_placement_count=('hash_author_id', 'count')
).reset_index()
data = pd.merge(data, offer_stats, on='hash_offer_id', how='left')

author_category_stats = data.groupby(['hash_author_id', 'category']).agg(
    author_category_avg_ctr=('ctr', 'mean')
).reset_index()
data = pd.merge(data, author_category_stats, on=['hash_author_id', 'category'], how='left')

data = pd.get_dummies(data, columns=['placement_format_x',
                                     'category',
                                     'is_published',
                                     'hash_seller_id',
                                     'hash_model_id'])

data = data.sample(n=5000)

print("Feature engineering complete. Final data shape:", data.shape)

print("Training the XGBoost model...")

features = [col for col in data.columns if col not in ['hash_author_id', 'hash_offer_id', 'clicks', 'views', 'ctr']]
X = data[features]
y = data['ctr']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    enable_categorical = True,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
print('Training completed!')

Starting feature engineering...


/var/folders/84/q5hn4ghd627930_h_hxhl4lh0000gn/T/ipykernel_30661/1743579147.py:19: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Feature engineering complete. Final data shape: (5000, 54617)
Training the XGBoost model...
Training completed!


In [61]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\nModel training complete. Test Set RMSE: {rmse:.6f}")


Model training complete. Test Set RMSE: 0.000784
